IEBS - Proyecto Final

Este notebook se encarga a traves de las APi de twitter buscar los tweets de los diferentes bancos a ser evaluados. Con esto se genera un archivo csv de los tweets y de los usuarios. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
import requests
from requests_oauthlib import OAuth1
import pandas as pd

from tweepy import OAuthHandler, API, Cursor
import json

import warnings
warnings.filterwarnings('ignore')

In [5]:
ckey = 'dHJRdt64HiFuM9Pd9JjfRKmaO'
csecret = 'ZeCqoUN9EntyQTWrQ1LjeuhRCcuPJsscQOhjtToR9o0WgolOPJ'

atoken = '38060754-NJ7TvzMaZGTbPqxN47R7Lo7nqonm92UnQ00gRcFLd'
asecret = 'Ce5lMmc97yKsloWxNT6MBCduFj8UcUQUXuWW39lz8Pezk'

Bearer = 'AAAAAAAAAAAAAAAAAAAAAHbEOwEAAAAAmFuxYzwGEgoSPj9UXJID8tJA4A4%3DdHHFgBO1XnM4MWI9Z40Tim28zBieDeTwPhU3EGMvFo6F6tvRcO'

In [6]:
auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

In [7]:
api = API(auth,
        wait_on_rate_limit=True,
        wait_on_rate_limit_notify=True)

# API con request

Request de prueba:
curl "https://api.twitter.com/2/users/2244994945/mentions?expansions=author_id&tweet.fields=conversation_id,lang&user.fields=created_at,entities&max_results=5" -H "Authorization: Bearer $BEARER_TOKEN"

Página: 
https://developer.twitter.com/en/docs/twitter-api/tweets/timelines/api-reference/get-users-id-mentions#tab1


In [173]:
headers = {'Authorization': f'Bearer {Bearer}'} # Variable de autorizador
url = f'https://api.twitter.com/2/users/by' # url de invocacion
bancos = '' # variable que guarda toda la respuesta

params = {'usernames': 'BBVA_espana,santander_es,caixabank,BancoSabadell,Bankinter',}

auth = OAuth1(ckey, csecret,
               atoken, asecret)

bancos = requests.get(url, params=params, headers=headers).json()
bancos

{'data': [{'id': '230123496',
   'name': 'BBVA en España',
   'username': 'BBVA_espana'},
  {'id': '1372470686', 'name': 'Santander España', 'username': 'santander_es'},
  {'id': '270429778', 'name': 'CaixaBank', 'username': 'caixabank'},
  {'id': '9980072', 'name': 'Banco Sabadell', 'username': 'BancoSabadell'},
  {'id': '86031609', 'name': 'Bankinter', 'username': 'Bankinter'}]}

In [151]:
#curl "https://api.twitter.com/2/tweets/search/all?query=from%3Atwitterdev%20new%20-is%3Aretweet&max_results=10" -H "Authorization: Bearer $BEARER_TOKEN"        

def buscar_tweets_mentions(mentions_id, cantidad_tweet=800):

    headers = {'Authorization': f'Bearer {Bearer}'} # Variable de autorizador
    url = f'https://api.twitter.com/2/users/{mentions_id}/mentions' # url de invocacion
    respuesta = '' # variable que guarda toda la respuesta
    usuarios = '' # variable que guarda los usuarios
    data = pd.DataFrame()
    params = {
        'expansions': 'author_id,geo.place_id',
        'tweet.fields': 'conversation_id,created_at,lang,public_metrics,possibly_sensitive,source',
        'max_results':100} 

    auth = OAuth1(ckey, csecret,
               atoken, asecret)
    
    for cantidad in range(int(cantidad_tweet/100)):
        respuesta = requests.get(url, params=params, headers=headers).json()
        data = data.append(respuesta['data'])
        usuarios = usuarios.append(respuesta['includes']['users'])
        params = {
            'expansions': 'author_id,geo.place_id',
            'tweet.fields': 'conversation_id,created_at,lang,public_metrics,possibly_sensitive,source',
            'pagination_token': respuesta['meta']['next_token'],
            'max_results':100}
    return data, usuarios

In [208]:
#  ruta_tweets = f'/content/drive/MyDrive/IEBS/Global Proyect/datos/{id}.csv'
#  ruta_users = f'/content/drive/MyDrive/IEBS/Global Proyect/datos/user-{id}.csv'

valores = bancos['data']
for i in valores:
  id = i['id']

  respuesta_funcion, usuarios_funcion = buscar_tweets_mentions(id)

  respuesta_funcion['created_at'] = pd.to_datetime(respuesta_funcion['created_at'])

  respuesta_funcion.to_csv(f'/content/drive/MyDrive/IEBS/Global Proyect/datos/{id}.csv')
  usuarios_funcion.to_csv(f'/content/drive/MyDrive/IEBS/Global Proyect/datos/user-{id}.csv')